In [4]:
import math

def relu(x, get_d=False):
    return x * (x > 0) if not get_d else 1.0 * (x >= 0)

def tanh(x, get_d=False):
    return math.tanh(x) if not get_d else 1 - math.tanh(x) ** 2

def sigmoid(x, get_d=False):
    return 1 / (1 + math.exp(-x)) if not get_d else sigmoid(x) * (1 - sigmoid(x))

In [2]:
import numpy as np

class Neuron:

    def __init__(self, n_in, n_w_per_edge, w_range=None):
        self.n_in = n_in
        self.n_w_per_edge = n_w_per_edge
        w_range = [-1, 1] if w_range is None else w_range
        self.w = np.random.uniform(w_range[0], w_range[-1], size=(self.n_in, self.n_w_per_edge))
        self.b = 0

        self.xin = None
        self.xmid = None
        self.xout = None

        self.dxout_dxmid = None
        self.dxout_db = None

        self.dxmid_dw = None
        self.dxmid_dxin = None

        self.dxout_dw = None
        self.dxout_dxin = None

        self.dl_dw = np.zeros((self.n_in, self.n_w_per_edge))
        self.dl_db = 0

    def __call__(self, xin):
        self.xin = np.array(xin)
        self.get_xmid()
        self.get_xout()

        self.get_dxout_dxmid()
        self.get_dxout_db()
        self.get_dxmid_dw()
        self.get_dmid_dxin()

        assert self.dxout_dxmid.shape == (self.n_in, )
        assert self.dxmid_dxin.shape == (self.n_in, )
        assert self.dxmid_dw.shape == (self.n_in, self.n_w_per_edge)

        self.get_dxout_dw()
        self.get_dxout_dxin()

        return self.xout

    def get_xmid(self):
        pass

    def get_xout(self):
        pass

    def get_dxout_dxmid(self):
        pass

    def get_dxout_db(self):
        pass

    def get_dxmid_dw(self):
        pass

    def get_dmid_dxin(self):
        pass

    def get_dxout_dw(self):
        self.dxout_dw = np.diag(self.dxout_dxmid) @ self.dxmid_dw

    def get_dxout_dxin(self):
        self.dxout_dxin = self.dxout_dxmid * self.dxmid_dxin

    def update_dl_dw_db(self, dl_dxout):
        self.dl_dw += self.dxout_dw * dl_dxout
        self.dl_db += self.dxout_db * dl_dxout

    def grad(self, lr):
        self.w -= lr * self.dl_dw
        self.b -= lr * self.dl_db
    




In [12]:
hello=np.array([[1,2,3], [4,5,6]])
bye = np.array([0.4, 0.5])
hello[:, 0] * bye

np.reshape(hello, (-1, 1))

array([[1],
       [2],
       [3],
       [4],
       [5],
       [6]])

In [15]:
class NeuronNN(Neuron):
    def __init__(self, n_in, w_range=None, activation=relu):
        super().__init__(self, n_in, n_w_per_edge=1, w_range=w_range)
        self.activation = activation
        self.activation_input = None

    def get_xmid(self):
        self.xmid = self.w[:, 0] * self.xin

    def get_xout(self):
        self.activation_input = sum(self.xmid.flatten()) + self.b
        self.xout = self.activation(self.activation_input, get_d=False)

    def get_dxout_dxmid(self):
        self.dxout_dxmid = self.activation(self.activation_input, get_d=True) * np.ones(self.n_in)

    def get_dxout_db(self):
        self.dxout_dxmid = self.activation(self.activation_input, get_d=True)

    def get_dxmid_dw(self):
        self.dxmid_dw = np.reshape(self.xin, (-1, 1))

    def get_dxmid_dxin(self):
        self.dxmid_dxin = self.w.flatten()
    

In [13]:
from scipy.interpolate import BSpline

def get_bsplines(x_bounds, n_fun, degree=3, **kwargs):
    grid_len = n_fun - degree + 1
    step = (x_bounds[1] - x_bounds[0]) / (grid_len - 1)
    edge_fun, edge_fun_d = {}, {}

    #SiLU bias function
    edge_fun[0] = lambda x: x / (1 + np.exp(-x))
    edge_fun_d[0] = lambda x: (1 + np.exp(-x) + x * np.exp(-x)) / np.power((1 + np.exp(-x)), 2)
    t = np.linspace(x_bounds[0] - degree * step, x_bounds[1] + degree * step, grid_len + 2 * degree)
    t[degree] = x_bounds[0]
    t[- degree - 1] = x_bounds[1]

    for i_spline in range(n_fun - 1):
        edge_fun[i_spline + 1] = BSpline.basis_element()
        edge_fun_d[i_spline + 1] = edge_fun[i_spline + 1].derivative()

    return edge_fun, edge_fun_d

In [ ]:
class NeuronKAN(Neuron):

    def __init__(self, n_in, n_w_per_edge, x_bounds, w_range=None, get_edge_f=get_bsplines, **kwargs):
        self.x_bounds = x_bounds
        super().__init__(n_in, n_w_per_edge, w_range)
        self.edge_f, self.edge_f_d = get_edge_f(x_bounds, n_w_per_edge, **kwargs)

    def get_xmid(self):
        # apply edge functions
        self.phi_x_mat = np.array([self.edge_f[b](self.xin) for b in self.edge_f]).T
        self.phi_x_mat[np.isnan(self.phi_x_mat)] = 0
        self.xmid = (self.w * self.phi_x_mat).sum(axis=1)

    def get_xout(self):
        # use tanh as activation function to avoid any updates of the spline grids
        self.xout = tanh(sum(self.xmid.flatten()), get_d=False)

    def get_dxout_dxmid(self):
        self.dxout_dxmid = tanh(sum(self.xmid.flatten()), get_d=True)

    def get_dxout_db(self):
        self.dxout_db = 0

    def get_dxmid_dw(self):
        self.dxmid_dw = self.phi_x_mat

    def get_dxmid_dxin(self):
        phi_x_d_mat = np.array([
            self.edge_f_der[b](self.xin)
            if self.edge_f_der[b](self.xin) is not None
            else 0 
            for b in self.edge_f
        ]).T # shape (n_in, n_w_per_edge)
        phi_x_d_mat[np.isnan(phi_x_d_mat)] = 0

        self.dxmid_dxin = (self.w * phi_x_d_mat).sum(axis=1)

In [16]:
from typing import Any


class FullyConnectedLayer:

    def __init__(self, n_in, n_out, neuron_class=NeuronNN, **kwargs) -> None:
        self.n_in = n_in
        self.n_out = n_out
        self.neurons = [neuron_class(n_in) if (kwargs == {}) else neuron_class(n_in, **kwargs) for n in range(n_out)]
        self.xin = None # input; shape (n_in, )
        self.xout = None # output; shape (n_out, )
        self.dl_dxin = None # dloss/dxin; shape (n_in, )
        self.zero_grad()

    def __call__(self, xin) -> Any:
        self.xin = xin
        self.xout = np.array([n(self.xin) for n in self.neurons])
        return self.xout

    def zero_grad(self, which=None):
        if which is None:
            which = ['xin', 'weights', 'bias']
        for w in which:
            if w == 'xin':
                self.dl_dxin = np.zeros(self.n_in)
            elif w == 'weights':
                for n in self.neurons:
                    n.dl_dw = np.zeros(self.n_in, self.neurons[0].n_w_per_edge)
            elif w == 'bias':
                for n in self.neurons:
                    n.dl_db = 0
            else:
                raise ValueError('input \'which\' value not recognized')

    def update_grad(self, dl_dxout):
        for i, dl_dxout_tmp in enumerate(dl_dxout):
            self.dl_dxin += self.neurons[i].dxout_dxin * dl_dxout_tmp
            self.neurons[i].update_dl_dw_db(dl_dxout_tmp)
        return self.dl_dxin

In [ ]:
from typing import Any


class Loss:

    def __init__(self, n_in) -> None:
        self.n_in = n_in
        self.y, self.dl_dy, self.loss, self.y_train = None, None, None, None

    def __call__(self, y, y_train) -> Any:
        # y: output of the net
        # y_train: ground truth
        self.y = np.array(y)
        self.y_train = y_train
        self.get_loss()
        self.get_dl_dy()

    def get_loss(self):
        pass

    def get_dl_dy(self):
        pass

class SquaredLoss(Loss):
    def get_loss(self):
        self.loss = np.mean(np.power(self.y - self.y_train, 2))

    def get_dl_dy(self):
        self.dl_dy = 2 * (self.y - self.y_train) / self.n_in

class CrossEntropyLoss(Loss):
    def get_loss(self):
        self.loss = - np.log(self.y[self.y_train[0]]) / sum(np.exp(self.y))

    def get_dl_dy(self):
        self.dl_dy = np.exp(self.y) / sum(np.exp(self.y))
        self.dl_dy[self.y_train] -= 1

In [ ]:
from typing import Any
from tqdm import tqdm

class FeedForward:

    def __init__(self, layer_len, eps=.0001, seed=None, loss=SquaredLoss, **kwargs) -> None:
        self.seed = np.random.randint(int(1e4)) if seed is None else int(seed)
        np.random.seed(self.seed)

        self.layer_len = layer_len
        self.eps = eps
        self.n_layers = len(self.layer_len) - 1
        self.layers = [FullyConnectedLayer(self.layer_len[i], self.layer_len[i + 1], **kwargs) for i in range(self.n_layers)]
        self.loss = loss(self.layer_len[-1])
        self.loss_hist = None

    def __call__(self, x) -> Any:
        xin = x
        for l in range(self.layers):
            xin = self.layers[l](xin)

        return xin
    
    def backprop(self):
        delta = self.layers[-1].update_grad(self.loss.dl_dy)
        for l in range(self.n_layers - 1)[::-1]:
            delta = self.layers[l].update_grad(delta)

    def grad_desc_par(self):
        for l in self.layers:
            for n in l.neurons:
                n.grad(self.eps)

    def train(self, x_train, y_train, n_iter_max=10000, loss_tol=.1):
        self.loss_hist = np.zeros(n_iter_max)
        x_train = np.array(x_train)
        y_train = np.array(y_train)

        
